In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
import os
import numpy as np
import datetime
import cv2
import time

In [ ]:
import tensorflow as tf
from tf_keras.layers import Input,Dense
from tf_keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D,Conv2DTranspose
from tf_keras.callbacks import ReduceLROnPlateau,EarlyStopping
from tf_keras.callbacks import LearningRateScheduler,ModelCheckpoint
import tf_keras.backend as K
from tf_keras.preprocessing.image import ImageDataGenerator
from tf_keras import layers

In [ ]:
BATCH_SIZE = 16
IMG_SIZE = (256,256)

train_dir = 'dataset/TBX11K/imgs/train/'
test_dir = 'dataset/TBX11K/imgs/test/'

In [ ]:
from dataset import define_data
from sklearn.preprocessing import LabelBinarizer
train = LabelBinarizer()

In [ ]:
train_data, train_y = define_data(train_dir,IMG_SIZE)
target_val = train.fit_transform(train_y)

In [ ]:

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(np.array(train_data, ),target_val,test_size=0.15,random_state=42)

In [ ]:
import tf_keras
from tf_keras.applications.mobilenet import MobileNet,preprocess_inputs
from tf_keras.applications.mobilenet_v2 import MobileNetV2,preprocess_inputs
from tf_keras.applications.resnet import ResNet50,preprocess_inputs
from tf_keras.applications.resnet import ResNet101
from tf_keras.applications.resnet import ResNet152
from tf_keras.applications.vgg19 import VGG19,preprocess_inputs
from tf_keras.applications.vgg16 import VGG16,preprocess_inputs

In [ ]:
class backboneModel:

    def __init__(self):
        self.models = {}

    def add_model(self,name,fn,args,head_fn):
        self.models[name] = {
            "fn":fn,
            "args":args,
            "head":head_fn
        }
    
    def build(self,name,input_shape,new_args=None):
        model = self.models[name]
        args = model["args"].copy()
        if new_args:
            args = new_args
        base_model = model["fn"](
            include_top=False,
            weights="imagenet",
            input_shape=INPUT_SHAPE,
            **args
        )
        full_model = model["head"](input_shape,base_model)
        return full_model

In [ ]:
back_net = MobileNetV2
preprocess = tf_keras.applications.mobilenetv2.preprocess_inputs
args = {
    "input_tensor": inputs,
    "weights": "imagenet",
    "include_top": False,
    "alpha": 0.35
    }

In [ ]:
std_args = {ResNet50.__name__:{
                "weights"="imagenet",
                "include_top"=False,
                "input_shape"=INPUT_SHAPE},
            ResNet101.__name__:{
                "weights"="imagenet",
                "include_top"=False,
                "input_shape"=INPUT_SHAPE},
            ResNet152.__name__:{
                "weights"="imagenet",
                "include_top"=False,
                "input_shape"=INPUT_SHAPE},
            "resnet18":{
                "weights"="imagenet",
                "include_top"=False,
                "input_shape"=INPUT_SHAPE},
            MobileNet.__name__:{
                "input_tensor": INPUT_SHAPE,
                "weights": "imagenet",
                "include_top": False,
                "alpha": 0.35
            },
            MobileNetV2.__name__:{
                "input_tensor": INPUT_SHAPE,
                "weights": "imagenet",
                "include_top": False,
                "alpha": 0.35
                }
            }
custom_heads = {
    ResNet50.__name__:resnet_model_head,
    ResNet101.__name__:resnet_model_head,
    ResNet152.__name__:resnet_model_head,
    "resnet18":resnet_model_head,
    MobileNet.__name__:mobilenet_model_head,
    MobileNetV2.__name__:mobilenet_model_head
}
model_farm = backboneModel()
model_farm.add_model(back_net.__name__,back_net,std_args[back_net.__name__],custom_heads[back_net.__name__])

In [ ]:
def mobilenet_model_head(input_shape,base_model):
    inputs = Input(shape=input_shape,name="input_image")
    x=base_model.get_layer('out_relu').output
    x=GlobalAveragePooling2D(name="gap")(x)
    output=Dense(3,activation="softmax")(x)
    return tf_keras.Model(inputs,output)

In [ ]:
model = model(input_shape=(256,256,3))
model.compile(optimizer=tf_keras.optimizers.Adam(learning_rate=0.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

#model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
            # loss=keras.losses.BinaryCrossentropy(),
            # metrics = [keras.metrics.AUC(from_logits=True)]
model.summary()

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
filepath=f'{back_net.__name__}/class_weights.{{epoch:02d}}-{{val_loss:.2f}}.weights.h5'
callback = [
    tf_keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.1, patience=3, verbose=1, min_lr=1e-8 ),
    tf_keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1),
    tf_keras.callbacks.ModelCheckpoint(
        filepath=filepath,
        save_best_only=True,save_weights_only=True,verbose=1)
           ]

In [ ]:
start_time = datetime.datetime.now()

history = model.fit(x=x_train, 
                    y = y_train,
                    validation_split=0.15,
                    epochs=15,
                    batch_size = 16,
                   callbacks=callback)

elapsed_time = datetime.datetime.now() - start_time
print('Cell ran in ' + str(elapsed_time.seconds) + ' seconds and ' + str(elapsed_time.microseconds) + ' microseconds.')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['train acc','validation acc'])
plt.ylabel('auc')
plt.xlabel('epochs')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history[ 'val_loss'])
plt.legend(['train loss','validation loss'])
plt.ylabel('loss')
plt.xlabel('epochs')
plt.show()

In [ ]:
import tf_keras


def model(input_shape):
    inputs = Input(shape=input_shape,name="input_image")
    base_model = back_net(input_tensor = inputs, weights="imagenet", include_top=False, alpha=0.35)
    x=base_model.output
    x=GlobalAveragePooling2D(name="gap")(x)
    output=Dense(3,activation="softmax")(x)
    return tf_keras.Model(inputs,output)

In [ ]:
model = model(input_shape=(256,256,3))
model.load_weights('MobileNetV2\class_weights.09-0.30.weights.h5')
model.compile(optimizer=tf_keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf_keras.losses.CategoricalCrossentropy(),
              metrics=[tf_keras.metrics.Accuracy()])

model.evaluate(x_test,y_test)

In [ ]:
predictions = model.predict(x_test)

In [ ]:
preds = np.zeros((y_test.shape[0]))
for i,p in enumerate(predictions):
    l = np.argmax(p)
    preds[i] = l

y = [np.argmax(yy) for yy in y_test]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y,preds))

In [ ]:
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
import matplotlib.pyplot as plt
cm = confusion_matrix(y,preds,labels=[0,1,2])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=["NORM","NON-TB","TB"])
disp.plot()
plt.savefig(f"{back_net.__name__}-confusionmatrix.png")
plt.show()